In [1]:
get_data('Inputs/HE set/128-20-NR-NY-NF-0.pickle')

NameError: name 'get_data' is not defined

In [16]:
img = k['X'][0][:,:,0]
print(img.shape)
img = img.reshape((64,192))
import numpy as np
import cv2
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


(192, 64)


# Extractions

In [ ]:
mono_word = []
bi_words = []
chars = []
files = os.listdir('Inputs/hin_corp_unicode')
for file in files:
    print(file)
    import codecs
    f = codecs.open('Inputs/hin_corp_unicode/'+file, encoding='utf-8')
    for line in f:
        line = line.strip('\n').strip('\r').strip(' ')
        if line != '':
            words = line.split(' ')
            for word in words:
                if word not in mono_word:
                    mono_word.append(word)
                    for char in list(word):
                        if char not in chars:
                            chars.append(char)
            for i in range(len(words)-1):
                string = words[i]+' '+words[i+1]
                if string not in bi_words:
                    bi_words.append(string)
dict = {'bi-grams':np.array(bi_words), 'mono-grams':np.array(mono_word), 'symbols':np.array(symbols)}
import pickle
pickle.dump( dict, open( "Inputs/words_hindi.pickle", "wb" ) )

In [154]:
str=''
x_l = ['0','1','2','3','4','5','6','7']
y_l = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
for x in x_l:
    for y in y_l:
        str+='u\'\\u09'+x+y+'\','
#k = 'u\'\\u09'+x+y+'\','

In [1]:
symbols = [u'\u0900',u'\u0901',u'\u0902',u'\u0903',u'\u0904',u'\u0905',u'\u0906',u'\u0907',u'\u0908',u'\u0909',u'\u090A',u'\u090B',u'\u090C',u'\u090D',u'\u090E',u'\u090F',u'\u0910',u'\u0911',u'\u0912',u'\u0913',u'\u0914',u'\u0915',u'\u0916',u'\u0917',u'\u0918',u'\u0919',u'\u091A',u'\u091B',u'\u091C',u'\u091D',u'\u091E',u'\u091F',u'\u0920',u'\u0921',u'\u0922',u'\u0923',u'\u0924',u'\u0925',u'\u0926',u'\u0927',u'\u0928',u'\u0929',u'\u092A',u'\u092B',u'\u092C',u'\u092D',u'\u092E',u'\u092F',u'\u0930',u'\u0931',u'\u0932',u'\u0933',u'\u0934',u'\u0935',u'\u0936',u'\u0937',u'\u0938',u'\u0939',u'\u093A',u'\u093B',u'\u093C',u'\u093D',u'\u093E',u'\u093F',u'\u0940',u'\u0941',u'\u0942',u'\u0943',u'\u0944',u'\u0945',u'\u0946',u'\u0947',u'\u0948',u'\u0949',u'\u094A',u'\u094B',u'\u094C',u'\u094D',u'\u094E',u'\u094F',u'\u0950',u'\u0951',u'\u0952',u'\u0953',u'\u0954',u'\u0955',u'\u0956',u'\u0957',u'\u0958',u'\u0959',u'\u095A',u'\u095B',u'\u095C',u'\u095D',u'\u095E',u'\u095F',u'\u0960',u'\u0961',u'\u0962',u'\u0963',u'\u0964',u'\u0965',u'\u0966',u'\u0967',u'\u0968',u'\u0969',u'\u096A',u'\u096B',u'\u096C',u'\u096D',u'\u096E',u'\u096F',u'\u0970',u'\u0971',u'\u0972',u'\u0973',u'\u0974',u'\u0975',u'\u0976',u'\u0977',u'\u0978',u'\u0979',u'\u097A',u'\u097B',u'\u097C',u'\u097D',u'\u097E',u'\u097F','!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-','.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':',';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G','H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T','U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '£', 'Â']

# Training Module

In [1]:
import keras.callbacks
#-------------------------------------Read Pickle data---------------------------------------------------
def get_data(file):
    import pickle
    with open(file,'rb') as f:
        return pickle.load(f)
#-------------------------------------Process the labels-----------------------------------------------
def process_source_strings(strings_array):
    source_strings=[]
    for string in strings_array:
        word = string.split('_')[1]
        source_strings.append(word)
    return source_strings

def shuffle_mats_or_lists(matrix_list, stop_ind=None):
        import numpy as np
        ret = []

        assert all([len(i) == len(matrix_list[0]) for i in matrix_list])    # All array length must be equal
        len_val = len(matrix_list[0])

        if stop_ind is None:
            stop_ind = len_val
        assert stop_ind <= len_val

        a = list(range(stop_ind))
        np.random.shuffle(a)

        a += list(range(stop_ind, len_val))

        for mat in matrix_list:
            if isinstance(mat, np.ndarray):
                ret.append(mat[a])
            elif isinstance(mat, list):
                ret.append([mat[i] for i in a])
            else:
                raise TypeError('`shuffle_mats_or_lists` only supports numpy.array and list objects.')

        return ret
    
def text_to_labels(symbol_string, text):
        ret = []

        for char in text:
            ret.append(symbol_string.find(char))

        return ret

def labels_to_text(labels):
    ret = []

    for c in labels:
        if c == len(symbol_string):               # CTC Blank
            ret.append("")
        else:
            ret.append(symbol_string[c])

    return "".join(ret)

def is_valid_string(temp_str):
    for char in temp_str:
        if char not in symbol_list:
            return False
    return True

def paint_it(text, w, h, input_path, ratio, rotate=False, y_traslation=False, multi_fonts=False, script='H'):

    from PIL import Image, ImageDraw, ImageFont
    from numpy import asarray
    import numpy as np
    from scipy import ndimage
    import os
    import math
    
    #ratio = w/h
    #-------------------Canvas-----------------
    R = np.random.randint(0,255)
    G = np.random.randint(0,255)
    B = np.random.randint(0,255)
    base_color = int(0.21*R+0.72*G+0.07*B)
    surface = Image.new("RGBA", (w, h), color=(R,G,B,255))

    trig=0
    while trig==0:
    
        #-------------------Font Selection--------------------------------------

        if script=='E':
            path = input_path+'/fonts'
            final_text=text
        elif script=='H':
            path = input_path+'/Hindi_fonts/UTF'
            length_text = len(text)
            final_text = ''
            i=0
            while i<length_text-1:
                if text[i+1]==u'\u093F':
                    final_text+=u'\u093F'
                    final_text+=text[i]
                    i+=2
                else:
                    final_text+=text[i]
                    i+=1
            if len(final_text)<len(text):
                final_text+=text[-1]
        elif script=='G':
            path = input_path+'/Gujarati_fonts'
            finak_text=text

        if multi_fonts:
            font_selected = np.random.choice(os.listdir(path))
        else:
            font_selected = np.random.choice(os.listdir(path)[0:3])

        #-------------------Rotation Angle--------------------------------------------
        rotation_angle=0
        if rotate:
            rotation_angle = np.random.randint(50)-25
        if rotation_angle<=0:
            rad = (rotation_angle+360)*3.14/180
        else:
            rad = rotation_angle*3.14/180


        #-----------------------Font Size---------------------------------------------------

        font_size = 1
        f = ImageFont.truetype(path+'/'+font_selected, font_size)
        text_width, text_height = f.getsize(text)
        if text_height==0:
            text_ratio=90
        else:
            text_ratio = text_width/text_height

        if text_ratio>=ratio:         #text width is more
            subjected_width = text_width*abs(math.cos(rad))+text_height*abs(math.sin(rad))
            while subjected_width <= w-5:
                font_size+=1
                f = ImageFont.truetype(path+'/'+font_selected, font_size)
                text_width, text_height = f.getsize(text)
                subjected_width = text_width*abs(math.cos(rad))+text_height*abs(math.sin(rad))
            font_size-=1

        else:                         # text height is more
            subjected_height = text_height*abs(math.cos(rad))+text_width*abs(math.sin(rad))
            while subjected_height <= h-5:
                font_size+=1
                f = ImageFont.truetype(path+'/'+font_selected, font_size)
                text_width, text_height = f.getsize(text)
                subjected_height = text_height*abs(math.cos(rad))+text_width*abs(math.sin(rad))
            font_size-=1

        # ----------------------------- height and width after rotation ---------------------------------
        f = ImageFont.truetype(path+'/'+font_selected, font_size)
        text_width, text_height = f.getsize(text)
        final_height = text_height*abs(math.cos(rad))+text_width*abs(math.sin(rad))
        final_width = text_width*abs(math.cos(rad))+text_height*abs(math.sin(rad))
        
        
        if final_width<w-1 and final_height<h-1:
            trig=1
            
    #------------------------------X-Y Trasnpose------------------------------------------------
    max_shift_x = w-final_width
    max_shift_y = h-final_height
    
    
    top_left_x = np.random.randint(0, int(max_shift_x))
    
    if y_traslation:
        top_left_y = np.random.randint(0, int(max_shift_y))
    else:
        top_left_y = 2    
        
    
    #--------------------------------Upper Canvas--------------------------------------------

    text_canvas = Image.new("RGBA", (int(final_width), int(final_height)), color=(R,G,B,255))
    draw = ImageDraw.Draw(text_canvas)
    
    difference = 0
    while difference==0:
        R_ = np.random.randint(0,255)
        G_ = np.random.randint(0,255)
        B_ = np.random.randint(0,255)
        font_color = int(0.21*R_+0.72*G_+0.07*B_)
        if abs(font_color-base_color)>=127:
            difference=1
    
    draw.text((0, 0), final_text, font=f, fill=(R_,G_,B_))
    
    #-------------------------------Text Rotation----------------------------------------------

    if rotate:                                                                   
        text_canvas = text_canvas.rotate(rotation_angle, expand=1)
  
    # -----------------------------Paste Canvas---------------------------------------------------

    sx, sy = text_canvas.size
    surface.paste(text_canvas, (top_left_x, top_left_y, top_left_x+sx, top_left_y+sy), text_canvas)
    
    #-----------------------------Binary of Image-----------------------------------------------

    surface = asarray(surface.convert('L'))
    surface = surface.astype(np.float32)/255

    #------------------------------Gaussian Noise------------------------------------------------

    severity = np.random.uniform(0, 0.6)
    blur = ndimage.gaussian_filter(np.random.randn(*surface.shape) * severity, 1)                        
    surface = (surface + blur)
    surface[surface > 1] = 1
    surface[surface <= 0] = 0
    
    #-------------------------------Inverse of image------------------------------------------------

    k = np.random.choice([0,1])
    if k==1:
        surface = 1-surface

    return np.array([surface])

class Visuals_manual(keras.callbacks.Callback):

    def __init__(self, run_name, test_func, inputs, outputs, num_display_words=8):
        
        import os
        self.test_func = test_func
        
        self.output_dir = os.path.join(OUTPUT_DIR, run_name)
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
            
        self.inputs = inputs
        self.outputs = outputs
        self.num_display_words = num_display_words
       

    def show_edit_distance(self, num):
        import editdistance
        num_left = num
        mean_norm_ed = 0.0
        mean_ed = 0.0
        word_err = 0
        word_total = 0
        char_err = 0
        char_total = 0
        batch_num=1
        
        while num_left > 0:
            word_batch = self.inputs[(batch_num-1)*32:batch_num*32]
            num_proc = min(word_batch['inputs'].shape[0], num_left)
            word_total+=num_proc
            decoded_res = decode_batch(self.test_func, word_batch['inputs'][0:num_proc])
            
            for j in range(num_proc):
                char_total+=len(decoded_res[j])
                if decoded_res[j] != word_batch['source_str'][j]:
                    word_err+=1
                    k=0
                    while k<min(len(decoded_res[j]),len(word_batch['source_str'][j])):
                        if decoded_res[j][k] != word_batch['source_str'][j][k]:
                            char_err+=1
                        k+=1
                edit_dist = editdistance.eval(decoded_res[j], word_batch['source_str'][j])
                mean_ed += float(edit_dist)
                mean_norm_ed += float(edit_dist)
                #mean_norm_ed += float(edit_dist)  /len(word_batch['source_str'][j])
            batch_num+=1
            num_left -= num_proc
            
        mean_norm_ed = mean_norm_ed / num
        mean_ed = mean_ed / num
        print('\nOut of %d samples:  Mean edit distance: %.3f Mean normalized edit distance: %0.3f' % (num, mean_ed, mean_norm_ed))
        print('char err = ',char_err,'/',char_total,'  -------  word err = ',word_err,'/',word_total)

    def on_epoch_end(self, epoch, logs={}):
        import os
        import pylab
        from keras import backend as K
        
        self.model.save_weights(os.path.join(self.output_dir, 'weights%02d.h5' % (epoch)))
        #self.show_edit_distance(256)
        
        word_batch = self.inputs
        res = decode_batch(self.test_func, word_batch['inputs'][0:self.num_display_words])
        
        if word_batch['inputs'][0].shape[0] <= 256:
            cols = 2
        else:
            cols = 1
            
        for i in range(self.num_display_words):
            pylab.subplot(self.num_display_words // cols, cols, i + 1)
            
            if K.image_data_format() == 'channels_first':
                the_input = word_batch['inputs'][i, 0, :, :]
            else:
                the_input = word_batch['inputs'][i, :, :, 0]
                
    #------------------------------------------------for new image set----------------------------------------------------
            the_input = np.flip(the_input,0)
    #---------------------------------------------------------------------------------------------------------------------
            pylab.imshow(the_input.T, cmap='Greys_r')
            pylab.xlabel('Truth = \'%s\'\nDecoded = \'%s\'' % (word_batch['source_str'][i], res[i]))
            print('Real = \'%s --  Pred = \'%s\'' % (word_batch['source_str'][i], res[i]))
        fig = pylab.gcf()
        fig.set_size_inches(10, 13)
        pylab.savefig(os.path.join(self.output_dir, 'e%02d.png' % (epoch)))
        pylab.close()
        
class Visuals(keras.callbacks.Callback):

    def __init__(self, run_name, test_func, text_img_gen, num_display_words=8):
        
        import os
        self.test_func = test_func
        
        self.output_dir = os.path.join(OUTPUT_DIR, run_name)
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
            
        self.text_img_gen = text_img_gen
        self.num_display_words = num_display_words
       

    def show_edit_distance(self, num):
        import editdistance
        num_left = num
        mean_norm_ed = 0.0
        mean_ed = 0.0
        word_err = 0
        word_total = 0
        char_err = 0
        char_total = 0
        
        while num_left > 0:
            word_batch = next(self.text_img_gen)[0]
            num_proc = min(word_batch['inputs'].shape[0], num_left)
            word_total+=num_proc
            decoded_res = decode_batch(self.test_func, word_batch['inputs'][0:num_proc])
            
            for j in range(num_proc):
                char_total+=len(decoded_res[j])
                if decoded_res[j] != word_batch['source_str'][j]:
                    word_err+=1
                    k=0
                    while k<min(len(decoded_res[j]),len(word_batch['source_str'][j])):
                        if decoded_res[j][k] != word_batch['source_str'][j][k]:
                            char_err+=1
                        k+=1
                edit_dist = editdistance.eval(decoded_res[j], word_batch['source_str'][j])
                mean_ed += float(edit_dist)
                mean_norm_ed += float(edit_dist)
                #mean_norm_ed += float(edit_dist)  /len(word_batch['source_str'][j])
                
            num_left -= num_proc
            
        mean_norm_ed = mean_norm_ed / num
        mean_ed = mean_ed / num
        print('\nOut of %d samples:  Mean edit distance: %.3f Mean normalized edit distance: %0.3f' % (num, mean_ed, mean_norm_ed))
        print('char err = ',char_err,'/',char_total,'  -------  word err = ',word_err,'/',word_total)

    def on_epoch_end(self, epoch, logs={}):
        import os
        import pylab
        from keras import backend as K
        
        self.model.save_weights(os.path.join(self.output_dir, 'weights%02d.h5' % (epoch)))
        self.show_edit_distance(256)
        
        word_batch = next(self.text_img_gen)[0]
        res = decode_batch(self.test_func, word_batch['inputs'][0:self.num_display_words])
        
        if word_batch['inputs'][0].shape[0] <= 256:
            cols = 2
        else:
            cols = 1
            
        for i in range(self.num_display_words):
            pylab.subplot(self.num_display_words // cols, cols, i + 1)
            
            if K.image_data_format() == 'channels_first':
                the_input = word_batch['inputs'][i, 0, :, :]
            else:
                the_input = word_batch['inputs'][i, :, :, 0]
                
            pylab.imshow(the_input.T, cmap='Greys_r')
            pylab.xlabel('Truth = \'%s\'\nDecoded = \'%s\'' % (word_batch['source_str'][i], res[i]))
            print('Real = \'%s --  Pred = \'%s\'' % (word_batch['source_str'][i], res[i]))
        fig = pylab.gcf()
        fig.set_size_inches(10, 13)
        pylab.savefig(os.path.join(self.output_dir, 'e%02d.png' % (epoch)))
        pylab.close()


C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class Text_Image_Feed_Generator(keras.callbacks.Callback):
    
#------------------------------------------------------Initialize Class-----------------------------------------------------
    
    def __init__(self, symbol_string, input_path, num_words, validation_words, mini_batch_size, image_width, image_height, shrink_factor, absolute_length=48):
        
        self.input_path = input_path
        self.symbol_string = symbol_string
        self.symbol_list = list(symbol_string)
        
        self.num_words = num_words
        self.validation_words = validation_words
        self.training_words = self.num_words-self.validation_words
        self.mini_batch_size = mini_batch_size
        
        self.image_width = image_width
        self.image_height = image_height
        self.shrink_factor = shrink_factor
        
        self.output_size = len(self.symbol_string)+1
        self.blank_label = len(self.symbol_string)
        
        self.max_len = self.max_words()
        self.min_len = 3
        
        self.absolute_length = absolute_length
    
#------------------------------------------------------Maximum length of string------------------------------------------------    
    
    def max_words(self):
        
        #import os
        #from PIL import ImageFont
        #fonts = os.listdir(self.input_path+'/Hindi_fonts/UTF')
        
        #max_width=0
        #selected_font=''
        '''for font in fonts:
            f = ImageFont.truetype(self.input_path+'/Hindi_fonts/UTF/'+font, int(self.image_height*0.4))
            w,h = f.getsize('Wii')
            if w>max_width:
                max_width=w
            selected_font=font
        i=0
        temp = 0
        f = ImageFont.truetype(self.input_path+'/Hindi_fonts/UTF/'+selected_font, int(self.image_height*0.4))
        while temp<int(self.image_width*1.6):
            temp,_ = f.getsize('WA'*i)
            i+=1'''
            
        return np.random.randint((self.image_width//16)-(self.image_width//32),self.image_width//16)

#---------------------------------------------------Generate Word list ---------------------------------------------------

    def generate_word_list(self):
        import os
        import codecs
        import pickle

        self.string_list = []
        
        self.Y_data = np.ones([self.num_words, self.absolute_length]) * -1    # Maximum threshold of words possible
        self.X_text = []                                                      # Actual Text Data
        self.Y_len = [0] * self.num_words                                     # Actual Length of Labels
        
        
        
        while len(self.string_list) < self.num_words:
            import pickle
            pickle_in = open(self.input_path+'/words_hindi.pickle','rb')
            data = pickle.load(pickle_in)
            
            mono = data['mono-grams']
            bi = data['bi-grams']
            np.random.shuffle(mono)
            np.random.shuffle(bi)

            for string in mono:
                if self.num_words<32000:
                    if len(self.string_list) < self.num_words:                        
                        if len(string) <= self.max_len and len(string)>=self.min_len: 
                            self.string_list.append(string)
                if self.num_words>32000:
                    if len(self.string_list) < 20000:                        
                        if len(string) <= self.max_len and len(string)>=self.min_len: 
                            self.string_list.append(string)
            print(len(self.string_list),' mono-grams-pooled')
            for string in bi:
                if len(self.string_list) < self.num_words:
                    if len(string) <= self.max_len and len(string)>=self.min_len: 
                        self.string_list.append(string)
            print(len(self.string_list),' bi-grams-pooled')  
            if len(self.string_list) < self.num_words:
                print(len(self.string_list), ' Words pulled, iterating again to achieve total of ', self.num_words)

        for i, word in enumerate(self.string_list):
            self.Y_len[i] = len(word)
            self.Y_data[i, 0:len(word)] = text_to_labels(self.symbol_string,word)
            self.X_text.append(word)
            
        self.Y_len = np.expand_dims(np.array(self.Y_len), 1)                             #shape=(self.Y_len,1)
         
        self.current_val_index = self.training_words
        self.current_train_index = 0
        
#-----------------------------------------------------batch Maker----------------------------------------------------

    def get_batch(self, start_index, size, isTrain):
        
        from keras import backend as K
        
        if K.image_data_format() == 'channels_first':
            X_data = np.ones([size, 1, self.image_width, self.image_height])
        else:
            X_data = np.ones([size, self.image_width, self.image_height, 1])
            

        labels = np.ones([size, self.absolute_length])
        
        input_length = np.zeros([size, 1])
        label_length = np.zeros([size, 1])
        
        source_strings = []
        
        for i in range(size):
            
            if isTrain and i > size - 4:                                                 # some random blank data
                
                if K.image_data_format() == 'channels_first':
                    X_data[i, 0, 0:self.image_width, :] = self.make_img('')[0, :, :].T
                else:
                    X_data[i, 0:self.image_width, :, 0] = self.make_img('',)[0, :, :].T
                    
                labels[i, 0] = self.blank_label
                
                label_length[i] = 1
                source_strings.append('')
                
            else:
                
                if K.image_data_format() == 'channels_first':
                    X_data[i, 0, 0:self.image_width, :] = self.make_img(self.X_text[start_index + i])[0, :, :].T
                else:
                    X_data[i, 0:self.image_width, :, 0] = self.make_img(self.X_text[start_index + i])[0, :, :].T
                    
                labels[i, :] = self.Y_data[start_index + i]
                label_length[i] = self.Y_len[start_index + i]
                source_strings.append(self.X_text[start_index + i])
                
            input_length[i] = (self.image_width // self.shrink_factor) - 2        # same for all inputs
                
        inputs = {'inputs': X_data, 'labels': labels, 'input_length': input_length, 'label_length': label_length, 'source_str': source_strings}
        outputs = {'ctc': np.zeros([size])}  # dummy
        return (inputs, outputs)
    
#-----------------------------------------------------------Training & Testing Batch-----------------------------------------
        
    def next_train(self):
        while 1:
            ret = self.get_batch(self.current_train_index, self.mini_batch_size, isTrain=True)
            self.current_train_index += self.mini_batch_size
            
            if self.current_train_index >= self.training_words:
                self.current_train_index = self.current_train_index % 32
                (self.X_text, self.Y_data, self.Y_len) = shuffle_mats_or_lists([self.X_text, self.Y_data, self.Y_len], self.training_words)
                
            yield ret
            
    def next_val(self):
        while 1:
            
            ret = self.get_batch(self.current_val_index, self.mini_batch_size, isTrain=False)
            self.current_val_index += self.mini_batch_size
            
            if self.current_val_index >= self.num_words:
                self.current_val_index = self.training_words + self.current_val_index % 32
                
            yield ret
            
#--------------------------------------------------------Callbacks------------------------------------------------------------

    def on_train_begin(self, logs={}):
        self.generate_word_list()
        self.make_img = lambda text: paint_it(text, self.image_width, self.image_height, self.input_path, self.image_width//self.image_height, rotate=False, y_traslation=False, multi_fonts=False)
        
    def on_epoch_begin(self, epoch, logs={}):
        (self.X_text, self.Y_data, self.Y_len) = shuffle_mats_or_lists([self.X_text, self.Y_data, self.Y_len])
        if 3 <= epoch < 6:
            self.make_img = lambda text: paint_it(text, self.image_width, self.image_height, self.input_path, self.image_width//self.image_height, rotate=False, y_traslation=True, multi_fonts=False, )
        elif 6 <= epoch < 9:
            self.make_img = lambda text: paint_it(text, self.image_width, self.image_height, self.input_path, self.image_width//self.image_height, rotate=False, y_traslation=True, multi_fonts=True)
        elif epoch >= 9:        
            self.make_img = lambda text: paint_it(text, self.image_width, self.image_height, self.input_path, self.image_width//self.image_height, rotate=True, y_traslation=True, multi_fonts=True)        

    

In [3]:
def get_x_y(file, image_width, shrink_factor, absolute_length=32):
    import numpy as np
    data = get_data(file)
    size = len(data['X'])
    
    X = data['X']/255
    source_strings = data['Y']#process_source_strings(data['Y'])
    
    input_length = np.zeros([size, 1])
    label_length = np.zeros([size, 1])
    labels = np.ones([size, absolute_length]) * -1
    
    for i, word in enumerate(source_strings):
        label_length[i] = len(word)
        labels[i, 0:len(word)] = text_to_labels(symbol_string, word)
        input_length[i] = (image_width // (shrink_factor)) - 2
                
    inputs = {'inputs': X, 'labels': labels, 'input_length': input_length, 'label_length': label_length, 'source_str': source_strings}
    outputs = {'ctc': np.zeros([size])}
    return inputs, outputs

In [10]:
def ctc_lambda_func(args):
    from keras import backend as K
    
    y_pred, labels, input_length, label_length = args
    #print(y_pred)
    y_pred = y_pred[:, 2:, :]
    
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)


# For a real OCR application, this should be beam search with a dictionary
# and language model.  For this example, best path is sufficient.

def decode_batch(test_func, word_batch):
    import itertools
    out = test_func([word_batch])[0]
    ret = []
    for j in range(out.shape[0]):
        out_best = list(np.argmax(out[j, 2:], 1))
        print(list(labels_to_text(out_best)))
        out_best = [k for k, g in itertools.groupby(out_best)]
        outstr = labels_to_text(out_best)
        ret.append(outstr)
    return ret

def fire(x, pipe, out):
    from keras.layers.convolutional import Conv2D
    from keras.layers.merge import add, concatenate
    x = Conv2D(pipe, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(x)# name=name+'-0-1x1'
    y_ = Conv2D(out//2, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(x)#, name=name+'-1-1x1'
    y__ = Conv2D(out//2, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(x)#, name=name+'-1-3x3'
    y = concatenate([y_, y__],axis=-1)
    return y

In [5]:
def train(model_name, start_epoch, end_epoch, image_width, image_height=64, num_samples=16000, split_ratio=0.1, mini_batch_size=32, training_method='Generator', input_dir='Inputs', file_name=None):# Method = Generator/Manual
    from keras import backend as K
    from keras.layers.convolutional import Conv2D, MaxPooling2D
    from keras.layers import Input, Dense, Activation, BatchNormalization
    from keras.layers import Reshape, Lambda, Dropout
    from keras.layers.merge import add, concatenate
    from keras.models import Model
    from keras.layers.recurrent import GRU, LSTM
    from keras.optimizers import SGD, Adam
    from keras.utils.data_utils import get_file
    from keras.preprocessing import image
    import keras.callbacks
    import os
    
    
    # Input Parameters
    validation_words = int(num_samples * split_ratio)

    # Network parameters
                    
    stride_size = 2
    time_dense_size = 32
    rnn_stack = 512
    shrink_factor = stride_size**3
    
    absolute_length=32

    if K.image_data_format() == 'channels_first':
        input_shape = (1, image_width, image_height)
    else:
        input_shape = (image_width, image_height, 1)
    #print(K.image_data_format())
    #input_shape = (1, image_width, image_height)

#----------------------------------------------------Model-----------------------------------------------------------
        
    input_data = Input(name='inputs', shape=input_shape, dtype='float32')
    
    inner0 = Conv2D(64, (3,3), padding='same', activation='relu')(input_data)
    inner1 = MaxPooling2D(2)(inner0)
    
    inner2 = Conv2D(128, (3,3), padding='same', activation='relu')(inner1)
    inner3 = MaxPooling2D(2)(inner2)
    
    inner4 = Conv2D(256, (3,3), padding='same', activation='linear')(inner3)
    inner5 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(inner4)
    inner6 = Activation('relu')(inner5)
    inner7 = Conv2D(256, (3,3), padding='same', activation='relu')(inner6)
    inner8 = MaxPooling2D(pool_size=(2, 2), padding='same')(inner7)
    
    inner9 = Conv2D(512, (3,3), padding='same', activation='linear')(inner8)
    inner10 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(inner9)
    inner11 = Activation('relu')(inner10)
    inner12 = Conv2D(512, (3,3), padding='same', activation='relu')(inner11)
    inner13 = MaxPooling2D(pool_size=(2, 2), strides=(1,2), padding='same')(inner12)
    
    inner14 = Conv2D(512, (2,2), padding='same', activation='linear')(inner13)
    inner15 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(inner14)
    inner16 = Activation('relu')(inner15)
    inner17 = MaxPooling2D(pool_size=(2, 2), strides=(1,2), padding='same')(inner16)
    
    inner18 = Conv2D(512, (2,2), strides=(1,2), padding='same', activation='linear')(inner17)
    inner19 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(inner18)
    inner20 = Activation('relu')(inner19)
    
    rnn_in_dims = (image_width // (shrink_factor), image_height*8 )
    
    inner21 = Reshape(target_shape=rnn_in_dims, name='reshape')(inner20)

    # cuts down input size going into RNN:
    #inner21 = Dense(256, activation='relu', name='dense1')(inner21)

    lstm_1f = LSTM(rnn_stack//2, return_sequences=True, kernel_initializer='he_normal', name='lstm_1f')(inner21)
    lstm_1b = LSTM(rnn_stack//2, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='lstm_1b')(inner21)
    lstm1_merged = concatenate([lstm_1f, lstm_1b])
    em1 = Dense(256)(lstm1_merged)
    
    lstm_2f = LSTM(rnn_stack//2, return_sequences=True, kernel_initializer='he_normal', name='lstm_2f')(em1)
    lstm_2b = LSTM(rnn_stack//2, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='lstm_2b')(em1)
    lstm2_merged = concatenate([lstm_2f, lstm_2b])
    em2 = Dense(len(symbol_string)+1)(lstm2_merged)
    y_pred = Activation('softmax', name='softmax')(em2)
    # transforms RNN output to character activations:
    #inner = Dense(len(symbol_string)+1, kernel_initializer='he_normal', name='dense2')(concatenate([lstm_2f, lstm_2b]))
    #y_pred = Activation('softmax', name='softmax')(inner)
    
    Model(inputs=input_data, outputs=y_pred).summary()

#---------------------------------------------------Got the outputs-------------------------------------------

    labels = Input(name='labels', shape=[absolute_length], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    
    # Keras doesn't currently support loss funcs with extra parameters so CTC loss is implemented in a lambda layer
    
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
    
    #adam = Adam(lr=0.025, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=1e-6)
    sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)   # clip the gradients

    model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)

    # the loss calc occurs elsewhere, so use a dummy lambda func for the loss
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=sgd)
    
    if start_epoch > 0:
        weight_file = os.path.join(OUTPUT_DIR, os.path.join(run_name, 'weights%02d.h5' % (start_epoch - 1)))
        model.load_weights(weight_file)
        
        
    test_func = K.function([input_data], [y_pred])
    
    
    if training_method=='Generator':
        Feed = Text_Image_Feed_Generator(symbol_string, input_dir, num_samples, validation_words, mini_batch_size, image_width, image_height, shrink_factor, absolute_length=absolute_length)
        
        viz_cb = Visuals(run_name, test_func, Feed.next_val())
    
    
        steps = (num_samples - validation_words) // mini_batch_size
        val_steps = validation_words // mini_batch_size

        model.fit_generator(generator=Feed.next_train(), steps_per_epoch=steps, initial_epoch=start_epoch, epochs=end_epoch,
                            validation_data=Feed.next_val(), validation_steps=val_steps,
                            callbacks=[viz_cb, Feed])
        
    elif training_method=='Manual':
        Feed=0
        i, o = get_x_y(file_name, image_width, shrink_factor)
        i_eval = {'inputs': i['inputs'][:1000], 'labels': i['labels'][:1000], 'input_length': i['input_length'][:1000], 'label_length': i['label_length'][:1000], 'source_str': i['source_str'][:1000]}
        o_eval = {'ctc': o['ctc'][:1000]}
        
        viz = Visuals_manual(run_name, test_func, i_eval, o_eval)
          
        model.fit(x=i, y=o, batch_size=mini_batch_size, epochs=end_epoch, initial_epoch=start_epoch, callbacks=[viz], 
                  validation_split=split_ratio, shuffle=True)
        
    else:
        raise ValueError('Please enter valid methon : Generator/Manual')
        
                               
    
    

    

In [6]:
def train(model_name, start_epoch, end_epoch, image_width, image_height=64, num_samples=16000, split_ratio=0.1, mini_batch_size=32, training_method='Generator', input_dir='Inputs', file_name=None):# Method = Generator/Manual
    from keras import backend as K
    from keras.layers.convolutional import Conv2D, MaxPooling2D
    from keras.layers import Input, Dense, Activation, BatchNormalization
    from keras.layers import Reshape, Lambda, Dropout
    from keras.layers.merge import add, concatenate
    from keras.models import Model
    from keras.layers.recurrent import GRU, LSTM
    from keras.optimizers import SGD, Adam
    from keras.utils.data_utils import get_file
    from keras.preprocessing import image
    import keras.callbacks
    import os
    
    
    # Input Parameters
    validation_words = int(num_samples * split_ratio)

    # Network parameters
                    
    stride_size = 2
    time_dense_size = 32
    rnn_stack = 512
    shrink_factor = stride_size**3
    
    absolute_length=32

    if K.image_data_format() == 'channels_first':
        input_shape = (1, image_width, image_height)
    else:
        input_shape = (image_width, image_height, 1)
    #print(K.image_data_format())
    #input_shape = (1, image_width, image_height)

#----------------------------------------------------Model-----------------------------------------------------------
        
    input_data = Input(name='inputs', shape=input_shape, dtype='float32')
    
    inner0 = Conv2D(64, (3,3), padding='same', activation='relu')(input_data)
    inner1 = MaxPooling2D(2)(inner0)
    
    inner2 = Conv2D(128, (3,3), padding='same', activation='relu')(inner1)
    inner3 = MaxPooling2D(2)(inner2)
    
    inner4 = Conv2D(256, (3,3), padding='same', activation='linear')(inner3)
    inner5 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(inner4)
    inner6 = Activation('relu')(inner5)
    inner7 = Conv2D(256, (3,3), padding='same', activation='relu')(inner6)
    inner8 = MaxPooling2D(pool_size=(2, 2), padding='same')(inner7)
    
    inner9 = Conv2D(512, (3,3), padding='same', activation='linear')(inner8)
    inner10 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(inner9)
    inner11 = Activation('relu')(inner10)
    inner12 = Conv2D(512, (3,3), padding='same', activation='relu')(inner11)
    inner13 = MaxPooling2D(pool_size=(2, 2), strides=(1,2), padding='same')(inner12)
    
    inner14 = Conv2D(512, (2,2), padding='same', activation='linear')(inner13)
    inner15 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(inner14)
    inner16 = Activation('relu')(inner15)
    inner17 = MaxPooling2D(pool_size=(2, 2), strides=(1,2), padding='same')(inner16)
    
    inner18 = Conv2D(512, (2,2), strides=(1,2), padding='same', activation='linear')(inner17)
    inner19 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(inner18)
    inner20 = Activation('relu')(inner19)
    
    rnn_in_dims = (image_width // (shrink_factor), image_height*8 )
    
    inner21 = Reshape(target_shape=rnn_in_dims, name='reshape')(inner20)

    # cuts down input size going into RNN:
    #inner21 = Dense(256, activation='relu', name='dense1')(inner21)

    lstm_1f = LSTM(rnn_stack, return_sequences=True, kernel_initializer='he_normal', name='lstm_1f')(inner21)
    lstm_1b = LSTM(rnn_stack, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='lstm_1b')(inner21)
    lstm1_merged = concatenate([lstm_1f, lstm_1b])
    em1 = Dense(512)(lstm1_merged)
    
    lstm_2f = LSTM(rnn_stack, return_sequences=True, kernel_initializer='he_normal', name='lstm_2f')(em1)
    lstm_2b = LSTM(rnn_stack, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='lstm_2b')(em1)
    lstm2_merged = concatenate([lstm_2f, lstm_2b])
    em2 = Dense(len(symbol_string)+1)(lstm2_merged)
    y_pred = Activation('softmax', name='softmax')(em2)
    # transforms RNN output to character activations:
    #inner = Dense(len(symbol_string)+1, kernel_initializer='he_normal', name='dense2')(concatenate([lstm_2f, lstm_2b]))
    #y_pred = Activation('softmax', name='softmax')(inner)
    
    Model(inputs=input_data, outputs=y_pred).summary()

#---------------------------------------------------Got the outputs-------------------------------------------

    labels = Input(name='labels', shape=[absolute_length], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    
    # Keras doesn't currently support loss funcs with extra parameters so CTC loss is implemented in a lambda layer
    
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
    
    #adam = Adam(lr=0.025, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=1e-6)
    sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)   # clip the gradients

    model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)

    # the loss calc occurs elsewhere, so use a dummy lambda func for the loss
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=sgd)
    
    if start_epoch > 0:
        weight_file = os.path.join(OUTPUT_DIR, os.path.join(run_name, 'weights%02d.h5' % (start_epoch - 1)))
        model.load_weights(weight_file)
        
        
    test_func = K.function([input_data], [y_pred])
    
    
    if training_method=='Generator':
        Feed = Text_Image_Feed_Generator(symbol_string, input_dir, num_samples, validation_words, mini_batch_size, image_width, image_height, shrink_factor, absolute_length=absolute_length)
        
        viz_cb = Visuals(run_name, test_func, Feed.next_val())
    
    
        steps = (num_samples - validation_words) // mini_batch_size
        val_steps = validation_words // mini_batch_size

        model.fit_generator(generator=Feed.next_train(), steps_per_epoch=steps, initial_epoch=start_epoch, epochs=end_epoch,
                            validation_data=Feed.next_val(), validation_steps=val_steps,
                            callbacks=[viz_cb, Feed])
        
    elif training_method=='Manual':
        Feed=0
        i, o = get_x_y(file_name, image_width, shrink_factor)
        i_eval = {'inputs': i['inputs'][:1000], 'labels': i['labels'][:1000], 'input_length': i['input_length'][:1000], 'label_length': i['label_length'][:1000], 'source_str': i['source_str'][:1000]}
        o_eval = {'ctc': o['ctc'][:1000]}
        
        viz = Visuals_manual(run_name, test_func, i_eval, o_eval)
          
        model.fit(x=i, y=o, batch_size=mini_batch_size, epochs=end_epoch, initial_epoch=start_epoch, callbacks=[viz], 
                  validation_split=split_ratio, shuffle=True)
        
    else:
        raise ValueError('Please enter valid methon : Generator/Manual')
        
                               
    
    

    

In [7]:
import datetime
import numpy as np
OUTPUT_DIR = 'image_hindi_ocr'                          
symbol_string =u'\u0900'u'\u0901'u'\u0902'u'\u0903'u'\u0904'u'\u0905'u'\u0906'u'\u0907'u'\u0908'u'\u0909'u'\u090A'u'\u090B'u'\u090C'u'\u090D'u'\u090E'u'\u090F'u'\u0910'u'\u0911'u'\u0912'u'\u0913'u'\u0914'u'\u0915'u'\u0916'u'\u0917'u'\u0918'u'\u0919'u'\u091A'u'\u091B'u'\u091C'u'\u091D'u'\u091E'u'\u091F'u'\u0920'u'\u0921'u'\u0922'u'\u0923'u'\u0924'u'\u0925'u'\u0926'u'\u0927'u'\u0928'u'\u0929'u'\u092A'u'\u092B'u'\u092C'u'\u092D'u'\u092E'u'\u092F'u'\u0930'u'\u0931'u'\u0932'u'\u0933'u'\u0934'u'\u0935'u'\u0936'u'\u0937'u'\u0938'u'\u0939'u'\u093A'u'\u093B'u'\u093C'u'\u093D'u'\u093E'u'\u093F'u'\u0940'u'\u0941'u'\u0942'u'\u0943'u'\u0944'u'\u0945'u'\u0946'u'\u0947'u'\u0948'u'\u0949'u'\u094A'u'\u094B'u'\u094C'u'\u094D'u'\u094E'u'\u094F'u'\u0950'u'\u0951'u'\u0952'u'\u0953'u'\u0954'u'\u0955'u'\u0956'u'\u0957'u'\u0958'u'\u0959'u'\u095A'u'\u095B'u'\u095C'u'\u095D'u'\u095E'u'\u095F'u'\u0960'u'\u0961'u'\u0962'u'\u0963'u'\u0964'u'\u0965'u'\u0966'u'\u0967'u'\u0968'u'\u0969'u'\u096A'u'\u096B'u'\u096C'u'\u096D'u'\u096E'u'\u096F'u'\u0970'u'\u0971'u'\u0972'u'\u0973'u'\u0974'u'\u0975'u'\u0976'u'\u0977'u'\u0978'u'\u0979'u'\u097A'u'\u097B'u'\u097C'u'\u097D'u'\u097E'u'\u097F'u'!"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~£Â '

In [8]:
import datetime
import numpy as np
OUTPUT_DIR = 'total_ocr'                          
symbol_string =u'\u0900'u'\u0901'u'\u0902'u'\u0903'u'\u0904'u'\u0905'u'\u0906'u'\u0907'u'\u0908'u'\u0909'u'\u090A'u'\u090B'u'\u090C'u'\u090D'u'\u090E'u'\u090F'u'\u0910'u'\u0911'u'\u0912'u'\u0913'u'\u0914'u'\u0915'u'\u0916'u'\u0917'u'\u0918'u'\u0919'u'\u091A'u'\u091B'u'\u091C'u'\u091D'u'\u091E'u'\u091F'u'\u0920'u'\u0921'u'\u0922'u'\u0923'u'\u0924'u'\u0925'u'\u0926'u'\u0927'u'\u0928'u'\u0929'u'\u092A'u'\u092B'u'\u092C'u'\u092D'u'\u092E'u'\u092F'u'\u0930'u'\u0931'u'\u0932'u'\u0933'u'\u0934'u'\u0935'u'\u0936'u'\u0937'u'\u0938'u'\u0939'u'\u093A'u'\u093B'u'\u093C'u'\u093D'u'\u093E'u'\u093F'u'\u0940'u'\u0941'u'\u0942'u'\u0943'u'\u0944'u'\u0945'u'\u0946'u'\u0947'u'\u0948'u'\u0949'u'\u094A'u'\u094B'u'\u094C'u'\u094D'u'\u094E'u'\u094F'u'\u0950'u'\u0951'u'\u0952'u'\u0953'u'\u0954'u'\u0955'u'\u0956'u'\u0957'u'\u0958'u'\u0959'u'\u095A'u'\u095B'u'\u095C'u'\u095D'u'\u095E'u'\u095F'u'\u0960'u'\u0961'u'\u0962'u'\u0963'u'\u0964'u'\u0965'u'\u0966'u'\u0967'u'\u0968'u'\u0969'u'\u096A'u'\u096B'u'\u096C'u'\u096D'u'\u096E'u'\u096F'u'\u0970'u'\u0971'u'\u0972'u'\u0973'u'\u0974'u'\u0975'u'\u0976'u'\u0977'u'\u0978'u'\u0979'u'\u097A'u'\u097B'u'\u097C'u'\u097D'u'\u097E'u'\u097F'u'!"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~£Â 'u'\u0A81'u'\u0A82'u'\u0A83'u'\u0A85'u'\u0A86'u'\u0A87'u'\u0A88'u'\u0A89'u'\u0A8A'u'\u0A8B'u'\u0A8C'u'\u0A8D'u'\u0A8F'u'\u0A90'u'\u0A91'u'\u0A93'u'\u0A94'u'\u0A95'u'\u0A96'u'\u0A97'u'\u0A98'u'\u0A99'u'\u0A9A'u'\u0A9B'u'\u0A9C'u'\u0A9D'u'\u0A9E'u'\u0A9F'u'\u0AA0'u'\u0AA1'u'\u0AA2'u'\u0AA3'u'\u0AA4'u'\u0AA5'u'\u0AA6'u'\u0AA7'u'\u0AA8'u'\u0AAA'u'\u0AAB'u'\u0AAC'u'\u0AAD'u'\u0AAE'u'\u0AAF'u'\u0AB0'u'\u0AB2'u'\u0AB3'u'\u0AB5'u'\u0AB6'u'\u0AB7'u'\u0AB8'u'\u0AB9'u'\u0ABC'u'\u0ABD'u'\u0ABE'u'\u0ABF'u'\u0AC0'u'\u0AC1'u'\u0AC2'u'\u0AC3'u'\u0AC4'u'\u0AC5'u'\u0AC7'u'\u0AC8'u'\u0AC9'u'\u0ACB'u'\u0ACC'u'\u0ACD'u'\u0AD0'u'\u0AE0'u'\u0AE1'u'\u0AE2'u'\u0AE3'u'\u0AE6'u'\u0AE7'u'\u0AE8'u'\u0AE9'u'\u0AEA'u'\u0AEB'u'\u0AEC'u'\u0AED'u'\u0AEE'u'\u0AEF'u'\u0AF0'u'\u0AF1'u'\u0AF9'

In [11]:
if __name__ == '__main__':
    #run_name = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')   #%Y-%m-%d-%H-%M-%S
    #run_name = '2018-06-25-11-39-38'
    train(run_name, 1, 2, 128, num_samples=320, mini_batch_size=32, training_method='Manual', file_name='C:/Users/Jay Chakalasiya/AnacondaProjects/OCR - Baidu/Inputs/HE Set/128-32000-NR-NY-NF-1.pickle')
    #train(run_name, 46, 47, 192, num_samples=32000, mini_batch_size=32, training_method='Manual', file_name='C:/Users/Jay Chakalasiya/AnacondaProjects/Ipython/OCR/Inputs/Wordset/192/file0Image_data-192-set-1.pickle')
    #train(run_name, 47, 48, 192, num_samples=32000, mini_batch_size=32, training_method='Manual', file_name='C:/Users/Jay Chakalasiya/AnacondaProjects/Ipython/OCR/Inputs/Wordset/192/file1Image_data-192-set-0.pickle')
    #train(run_name, 48, 49, 192, num_samples=32000, mini_batch_size=32, training_method='Manual', file_name='C:/Users/Jay Chakalasiya/AnacondaProjects/Ipython/OCR/Inputs/Wordset/192/file1Image_data-192-set-1.pickle')
    #train(run_name, 49, 50, 192, num_samples=32000, mini_batch_size=32, training_method='Manual', file_name='C:/Users/Jay Chakalasiya/AnacondaProjects/Ipython/OCR/Inputs/Wordset/192/file2Image_data-192-set-0.pickle')
    #train(run_name, 50, 51, 192, num_samples=32000, mini_batch_size=32, training_method='Manual', file_name='C:/Users/Jay Chakalasiya/AnacondaProjects/Ipython/OCR/Inputs/Wordset/192/file2Image_data-192-set-1.pickle')
    

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 128, 64, 64)  640         inputs[0][0]                     
__________________________________________________________________________________________________
max_pooling2d_11 (MaxPooling2D) (None, 64, 32, 64)   0           conv2d_17[0][0]                  
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 64, 32, 128)  73856       max_pooling2d_11[0][0]           
__________________________________________________________________________________________________
max_poolin

In [13]:
abs(int(-5))

5

In [25]:
str=''
x_l = ['8','9','A','B','C','D','E','F']
y_l = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
for x in x_l:
    for y in y_l:
        str+='u\'\\u0A'+x+y+'\','
#k = 'u\'\\u09'+x+y+'\','


In [26]:
str

"u'\\u0A80',u'\\u0A81',u'\\u0A82',u'\\u0A83',u'\\u0A84',u'\\u0A85',u'\\u0A86',u'\\u0A87',u'\\u0A88',u'\\u0A89',u'\\u0A8A',u'\\u0A8B',u'\\u0A8C',u'\\u0A8D',u'\\u0A8E',u'\\u0A8F',u'\\u0A90',u'\\u0A91',u'\\u0A92',u'\\u0A93',u'\\u0A94',u'\\u0A95',u'\\u0A96',u'\\u0A97',u'\\u0A98',u'\\u0A99',u'\\u0A9A',u'\\u0A9B',u'\\u0A9C',u'\\u0A9D',u'\\u0A9E',u'\\u0A9F',u'\\u0AA0',u'\\u0AA1',u'\\u0AA2',u'\\u0AA3',u'\\u0AA4',u'\\u0AA5',u'\\u0AA6',u'\\u0AA7',u'\\u0AA8',u'\\u0AA9',u'\\u0AAA',u'\\u0AAB',u'\\u0AAC',u'\\u0AAD',u'\\u0AAE',u'\\u0AAF',u'\\u0AB0',u'\\u0AB1',u'\\u0AB2',u'\\u0AB3',u'\\u0AB4',u'\\u0AB5',u'\\u0AB6',u'\\u0AB7',u'\\u0AB8',u'\\u0AB9',u'\\u0ABA',u'\\u0ABB',u'\\u0ABC',u'\\u0ABD',u'\\u0ABE',u'\\u0ABF',u'\\u0AC0',u'\\u0AC1',u'\\u0AC2',u'\\u0AC3',u'\\u0AC4',u'\\u0AC5',u'\\u0AC6',u'\\u0AC7',u'\\u0AC8',u'\\u0AC9',u'\\u0ACA',u'\\u0ACB',u'\\u0ACC',u'\\u0ACD',u'\\u0ACE',u'\\u0ACF',u'\\u0AD0',u'\\u0AD1',u'\\u0AD2',u'\\u0AD3',u'\\u0AD4',u'\\u0AD5',u'\\u0AD6',u'\\u0AD7',u'\\u0AD8',u'\\u0AD9',u'\\u0ADA

In [29]:
u'\u0A81'u'\u0A82'u'\u0A83'u'\u0A85'u'\u0A86'u'\u0A87'u'\u0A88'u'\u0A89'u'\u0A8A'u'\u0A8B'u'\u0A8C'u'\u0A8D'u'\u0A8F'u'\u0A90'u'\u0A91'u'\u0A93'u'\u0A94'u'\u0A95'u'\u0A96'u'\u0A97'u'\u0A98'u'\u0A99'u'\u0A9A'u'\u0A9B'u'\u0A9C'u'\u0A9D'u'\u0A9E'u'\u0A9F'u'\u0AA0'u'\u0AA1'u'\u0AA2'u'\u0AA3'u'\u0AA4'u'\u0AA5'u'\u0AA6'u'\u0AA7'u'\u0AA8'u'\u0AAA'u'\u0AAB'u'\u0AAC'u'\u0AAD'u'\u0AAE'u'\u0AAF'u'\u0AB0'u'\u0AB2'u'\u0AB3'u'\u0AB5'u'\u0AB6'u'\u0AB7'u'\u0AB8'u'\u0AB9'u'\u0ABC'u'\u0ABD'u'\u0ABE'u'\u0ABF'u'\u0AC0'u'\u0AC1'u'\u0AC2'u'\u0AC3'u'\u0AC4'u'\u0AC5'u'\u0AC7'u'\u0AC8'u'\u0AC9'u'\u0ACB'u'\u0ACC'u'\u0ACD'u'\u0AD0'u'\u0AE0'u'\u0AE1'u'\u0AE2'u'\u0AE3'u'\u0AE6'u'\u0AE7'u'\u0AE8'u'\u0AE9'u'\u0AEA'u'\u0AEB'u'\u0AEC'u'\u0AED'u'\u0AEE'u'\u0AEF'u'\u0AF0'u'\u0AF1'u'\u0AF9'

'ઁંઃઅઆઇઈઉઊઋઌઍએઐઑઓઔકખગઘઙચછજઝઞટઠડઢણતથદધનપફબભમયરલળવશષસહ઼ઽાિીુૂૃૄૅેૈૉોૌ્ૐૠૡૢૣ૦૧૨૩૪૫૬૭૮૯૰૱ૹ'

In [58]:
k = u'\u092E'u'\u093A'

In [82]:
img = paint_it(k, 64, 64, 'Inputs', True, True, True)

In [83]:
img = img.reshape((64,64))
import numpy as np
import cv2
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [61]:
k[0]

'म'

In [60]:
k[1]

'ऺ'

In [59]:
k

'मऺ'